In [1]:
import copy
import datetime as dt
import json
import operator
import os

import iso8601
import pandas as pd

import bb_binary.parsing as bbb_p
import bb_videos_archiv.core as bb_va

In [2]:
def toggle(param):
    if param == 'left':
        return 'right'
    elif param == 'right':
        return 'left'
    elif param ==  0:
        return 1
    elif param == 1:
        return 0
    elif param == 2:
        return 3
    elif param == 3:
        return 2
    else:
        raise ValueError("Just the following values are accepted 'left', 'right', 0, 1, 2 and 3.")

In [3]:
def get_all_video_names_sorted(path):
    """Dies gibt ein dict zurück, welches als keys 'Cam_0' bis 'Cam_3' enthält.
    
    Unter jedem key ist eine geordnete liste aller filenames enthalten.
    """
    video_files = {}
    for __,__,files in os.walk(path):
        for file in files:
            if file.endswith('.mkv'):
                cam_id, __, __ = bbb_p.parse_video_fname(file)

                cam_id_str = 'Cam_{id}'.format(id=cam_id)
                if cam_id_str not in video_files:
                    video_files[cam_id_str] = []

                video_files[cam_id_str].append(file)

    for key in video_files.keys():
        video_files[key].sort()
    return video_files

In [4]:
PATHS_JSON_INTPUT = '05c_Cam_{cam_id}_intervals_ecc_stable_join_man.json'
DOC_PATH = '../docs'
ROOT_DIR = './videos_proxy'
YEAR = '2016'
THOLD = dt.timedelta(seconds=90)

SIDE_A_LEFT_CAM = 0
SIDE_A_RIGHT_CAM =1
SIDE_A_OUTPUT = '06_Cam_01_intervals_pair.json'

SIDE_B_LEFT_CAM = 2
SIDE_B_RIGHT_CAM = 3
SIDE_B_OUTPUT = '06_Cam_23_intervals_pair.json'

VIDEO_FILE_NAMES = get_all_video_names_sorted(ROOT_DIR)

In [5]:
def read_n_parse_json(path):
    df = pd.read_json(path)
    df['cam_id'] = df.apply(lambda row: bbb_p.parse_video_fname(row['start_video_name'])[0], axis=1)
    df['ts_start'] = df.apply(lambda row: bbb_p.parse_video_fname(row['start_video_name'])[1], axis=1)
    df['ts_end'] = df.apply(lambda row: bbb_p.parse_video_fname(row['end_video_name'])[2], axis=1)
    return df

# Load intervals and parse needed values.

In [6]:
dataframes = [] 
for i in range(4):
    path = PATHS_JSON_INTPUT.format(cam_id=i)
    dataframes.append(read_n_parse_json(path))
df = pd.concat(dataframes, axis=0, ignore_index=True)
df = df.sort_values(by='ts_start', ascending=False).reset_index(drop=True)

In [7]:
df[:4]

,end_video_name,id,info,stable,start_video_name,cam_id,ts_start,ts_end
0,Cam_0_2016-09-19T08:40:12.278788Z--2016-09-19T...,19,last interval,True,Cam_0_2016-09-14T15:19:20.096810Z--2016-09-14T...,0,2016-09-14 15:19:20.096810+00:00,2016-09-19 08:45:52.125778+00:00
1,Cam_2_2016-09-19T06:39:06.541317Z--2016-09-19T...,23,manuell connected,True,Cam_2_2016-08-30T14:25:42.806085Z--2016-08-30T...,2,2016-08-30 14:25:42.806085+00:00,2016-09-19 06:44:53.695644+00:00
2,Cam_3_2016-09-19T06:40:29.591957Z--2016-09-19T...,27,manuell connected,True,Cam_3_2016-08-23T07:12:20.761501Z--2016-08-23T...,3,2016-08-23 07:12:20.761501+00:00,2016-09-19 06:46:16.414852+00:00
3,Cam_1_2016-09-19T08:40:14.275312Z--2016-09-19T...,21,last interval,True,Cam_1_2016-08-21T16:46:43.637656Z--2016-08-21T...,1,2016-08-21 16:46:43.637656+00:00,2016-09-19 08:45:54.119255+00:00


# Join start timestamps of intervals from the left and right side of comb

In [8]:
def get_start_of_shared_intervals(sorted_data, cam_id_left, cam_id_right, thold):
    """This function gets all start timestamps from the 'left' and 'right' side of the comb.
    
    If timestamps of the 'left' and 'right' side of the comb are close to each other (`< thold`),
    then they are considered to be associated with the same interval.
    
    Args:
        sorted_data (df): see above
        cam_id_left (int):
        cam_id_right (int):
        thold (datetime.timedelta): see function description.
    
    Return:
        json (see below)
    """
    concat = False
    shared_intervals = []
    memory = None
    id_sh = 0
    for i, (idx_i, row_older) in enumerate(sorted_data.iterrows()):

        # Intervall besteht aus start und entpunkt, als gesamte Zeitangabe und dann jeweils die entsprechendenden Videos
        # für jede Kamera
        shared_interval = {}
        export_keys = ['stable', 'id', 'info', 'start_video_name', 'ts_start', 'cam_id']
        
        side = 'left' if row_older['cam_id'] == cam_id_left else 'right'

        shared_interval[side] = {}
        for key in export_keys:
            shared_interval[side][key] = row_older[key]
        shared_interval[toggle(side)] = memory

        if memory is not None:
            memory = None
            shared_interval['id'] = id_sh
            id_sh += 1
            shared_intervals.append(shared_interval)
            continue

        row_younger = sorted_data.iloc[i+1]

        if row_younger['cam_id'] == row_older['cam_id']: 
            shared_interval['id'] = id_sh
            id_sh += 1
            shared_intervals.append(shared_interval)
        else:
            if (row_older['ts_start'] - row_younger['ts_start']) < thold:
                memory = {}
                for key in export_keys:
                    memory[key] = row_older[key]
            else:
                shared_interval['id'] = id_sh
                id_sh += 1
                shared_intervals.append(shared_interval)
    return shared_intervals

`shared_intervals` looks like this:
```json
[..
{'id': 5,
  'left': {'cam_id': 0,
   'id': 17,
   'info': 'manuell connected',
   'stable': True,
   'start_video_name': 'Cam_0_2016-08-03T13:11:49.717921Z--2016-08-03T13:17:29.562534Z.mkv',
   'ts_start': Timestamp('2016-08-03 13:11:49.717921+0000', tz='<iso8601.Utc>')},
  'right': {'cam_id': 1,
   'id': 18,
   'info': 'ecc stable',
   'stable': True,
   'start_video_name': 'Cam_1_2016-08-03T13:11:48.360014Z--2016-08-03T13:17:28.233244Z.mkv',
   'ts_start': Timestamp('2016-08-03 13:11:48.360014+0000', tz='<iso8601.Utc>')}},
 {'id': 6,
  'left': None,
  'right': {'cam_id': 1,
   'id': 17,
   'info': 'manuell connected',
   'stable': True,
   'start_video_name': 'Cam_1_2016-08-03T11:02:12.211250Z--2016-08-03T11:06:49.270722Z.mkv',
   'ts_start': Timestamp('2016-08-03 11:02:12.211250+0000', tz='<iso8601.Utc>')}},
   ..]
```

-------

# Find closest video
Since the intervals do not contain a start video for each side, we determine these first.

In [9]:
def get_closest_video(ts, sorted_video_files, th):
    """Find the closest video to the given timestamp `ts`.
    
    If the timestamp `ts` is near the end of an video `< th`,
    then the next video will be returned.
    
    Args:
        ts(datetime):
        sorted_video_files(list):
        th(datetime.timedelta): 
    
    Returns:
        video_file (string)
    """
    time_delta = dt.timedelta(seconds=th)
    for i, video in enumerate(sorted_video_files):
        __, ts_start, ts_end = bbb_p.parse_video_fname(video)
        if ts_start < ts:
            if ts < ts_end:
                if ts_end - ts < time_delta and (i < len(sorted_video_files) -1):
                    return sorted_video_files[i+1]
                else:
                    return video
            else:
                continue
        else:
            return video

In [10]:
def extend_closest_videos(intervals):
    """Determines for every shared interval in `intervals` the start_video of the left or right side.
    
    Args:
        intervals(json)
    
    Returns:
        (json) extended with `start_video_name` for each side.
    """
    for i, interval in enumerate(intervals):
        for key in ['left', 'right']:
            if interval[key] is None:
                exist_key = toggle(key)
                non_exist_cam = toggle(interval[exist_key]['cam_id'])
                ts = interval[exist_key]['ts_start']
                ts = ts.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
                ts = iso8601.parse_date(ts)
                cam_id_str = 'Cam_{id}'.format(id=non_exist_cam)
                video = get_closest_video(ts, VIDEO_FILE_NAMES[cam_id_str], 5)
                
                __, ts_start_video, __= bbb_p.parse_fname(video)
                time_delta = dt.timedelta(minutes=30)
                
                if (abs(ts_start_video-ts) < time_delta):
                    interval[key] = {}
                    interval[key]['start_video_name'] = video
                    interval[key]['stable'] = intervals[i-1][key]['stable']
                    interval[key]['cam_id'], interval[key]['ts_start']= bbb_p.parse_video_fname(video)[:2]
                    interval[key]['info'] = 'closest video'
    return intervals

`extended_shared_intervals` now looks like this (compare it to the above `shared_intervals`):

```json
[
...
{'id': 6,
  'left': {'cam_id': 0,
   'info': 'closest video',
   'stable': True,
   'start_video_name': 'Cam_0_2016-08-03T11:02:12.547517Z--2016-08-03T11:06:49.270713Z.mkv',
   'ts_start': datetime.datetime(2016, 8, 3, 11, 2, 12, 547517, tzinfo=<iso8601.Utc>)},
  'right': {'cam_id': 1,
   'id': 17,
   'info': 'manuell connected',
   'stable': True,
   'start_video_name': 'Cam_1_2016-08-03T11:02:12.211250Z--2016-08-03T11:06:49.270722Z.mkv',
   'ts_start': Timestamp('2016-08-03 11:02:12.211250+0000', tz='<iso8601.Utc>')}},
 ...
]
```
In this case the `left` side is extended.

# Determine the end_video of every interval per side and mark intervals as stable.

In [11]:
def get_stable_info(df, cam_id, ts_start, ts_end):
    """Retrun the information if the interval from `ts_start` to `ts_end` is stable.
    
    df (df): dataframe with all initial intervals, which contains the information if they are stable or not.
    cam_id (int): Id of the camera
    ts_start (datetime): Start of the interval.
    ts_end (datetime): End of the interval.
    """
    result = df[((df.cam_id ==cam_id) & (df.ts_start <= ts_start) & (ts_end <= df.ts_end))]
    if len(result) == 1:
        return bool(result['stable'].values[0])
    else:
        print(ts_start)
        print(ts_end)
        print(cam_id)
        return True
        raise Exception('Es gibt mehr als ein Interval in dem dieses "beschnitte" Interval liegt, das sollte nicht möglich sein!')

In [12]:
def extend_end_video_name(intervals, video_files):
    """Extend `interval` with the information about the last video of a interval per side.
    
    intervals(json):
    video_files(list):
    """
    for idx, interval in enumerate(intervals):
        for key in ['left', 'right']:
            
            if interval[key] is not None:
                cam_id_str = 'Cam_{id}'.format(id=interval[key]['cam_id'])
                curr_idx = idx + 1
                # falls das folgende Interval leer ist 
                while curr_idx  < len(intervals) and intervals[curr_idx][key] is None:
                    curr_idx = curr_idx + 1

                if curr_idx  < len(intervals) and intervals[curr_idx][key] is not None:
                    next_start_video = intervals[curr_idx][key]['start_video_name']
                    next_idx = video_files[cam_id_str].index(next_start_video)
                    end_idx = next_idx - 1
                    interval[key]['end_video_name'] = video_files[cam_id_str][end_idx]
                    __, __, interval[key]['ts_end'] = bbb_p.parse_video_fname(interval[key]['end_video_name'])
                    interval[key]['stable'] = get_stable_info(df, interval[key]['cam_id'],interval[key]['ts_start'], interval[key]['ts_end'])
                if curr_idx == len(intervals):
                    interval[key]['end_video_name'] = video_files[cam_id_str][-1]
                    __, __, interval[key]['ts_end'] = bbb_p.parse_video_fname(interval[key]['end_video_name'])
                    interval[key]['stable'] = get_stable_info(df, interval[key]['cam_id'],interval[key]['ts_start'], interval[key]['ts_end'])

`shared_intervals_valid_json_01`
```json
[
...
{'id': 6,
  'left': {'cam_id': 0,
   'end_video_name': 'Cam_0_2016-08-03T11:14:10.077586Z--2016-08-03T11:17:04.485231Z.mkv',
   'info': 'closest video',
   'stable': True,
   'start_video_name': 'Cam_0_2016-08-03T11:02:12.547517Z--2016-08-03T11:06:49.270713Z.mkv',
   'ts_end': datetime.datetime(2016, 8, 3, 11, 17, 4, 485231, tzinfo=<iso8601.Utc>),
   'ts_start': datetime.datetime(2016, 8, 3, 11, 2, 12, 547517, tzinfo=<iso8601.Utc>)},
  'right': {'cam_id': 1,
   'end_video_name': 'Cam_1_2016-08-03T11:14:10.077634Z--2016-08-03T11:17:04.485290Z.mkv',
   'id': 17,
   'info': 'manuell connected',
   'stable': True,
   'start_video_name': 'Cam_1_2016-08-03T11:02:12.211250Z--2016-08-03T11:06:49.270722Z.mkv',
   'ts_end': datetime.datetime(2016, 8, 3, 11, 17, 4, 485290, tzinfo=<iso8601.Utc>),
   'ts_start': Timestamp('2016-08-03 11:02:12.211250+0000', tz='<iso8601.Utc>')}},
...
]
```

# Convert to valid json (no datetime)

In [13]:
def convert_2_valid_json(shared_intervals):
    shared_intervals_valid_json = []
    for interval in shared_intervals:
        valid_interval = copy.deepcopy(interval)
        valid_interval['id'] = len(shared_intervals) - valid_interval['id'] - 1
        for key in ['left', 'right']:
            if valid_interval[key] is not None:
                for key_ts in ['ts_start', 'ts_end']:
                    if key_ts in interval[key]:
                        valid_interval[key][key_ts] = interval[key][key_ts].strftime('%Y-%m-%dT%H:%M:%S.%fZ')

        for key_ts in ['ts_start', 'ts_end']:
            if key_ts in interval:
                valid_interval[key_ts] = interval[key_ts].strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        shared_intervals_valid_json.append(valid_interval)
    return shared_intervals_valid_json

# Determine shared start and end of intervals

In [14]:
def extend_overall_interval(intervals):
    """Bestimmt Anfangs und Endzeitpunkt des gemeinsamen Intervals und markiert ein gemeinsames
    Interval als stable, wenn beide Videos stable sind."""
    def _set_interval_key(interval, key, relate):
        if interval['left'] is not None and interval['right'] is not None:
            if relate(interval['left'][key], interval['right'][key]):
                interval[key] = interval['left'][key]
            else:
                interval[key] = interval['right'][key]
        else:
            if interval['left'] is None:
                interval[key] = interval['right'][key]
            else:
                interval[key] = interval['left'][key]

    for i, interval in enumerate(intervals):
        _set_interval_key(interval, 'ts_start', operator.lt)
        _set_interval_key(interval, 'ts_end', operator.gt)
        if interval['left'] is None:
            interval['stable'] = interval['right']['stable']
        elif interval['right'] is None:
            interval['stable'] = interval['left']['stable']
        else:
            interval['stable'] = interval['left']['stable'] and interval['right']['stable']

converted_01
```json
[...
{'id': 6,
  'left': {'cam_id': 0,
   'end_video_name': 'Cam_0_2016-07-27T14:59:16.877336Z--2016-07-27T15:04:56.723083Z.mkv',
   'id': 6,
   'info': 'ecc double movement',
   'stable': False,
   'start_video_name': 'Cam_0_2016-07-27T14:59:16.877336Z--2016-07-27T15:04:56.723083Z.mkv',
   'ts_end': '2016-07-27T15:04:56.723083Z',
   'ts_start': '2016-07-27T14:59:16.877336Z'},
  'right': {'cam_id': 1,
   'end_video_name': 'Cam_1_2016-07-27T14:59:17.210304Z--2016-07-27T15:04:57.056134Z.mkv',
   'id': 6,
   'info': 'ecc double movement',
   'stable': False,
   'start_video_name': 'Cam_1_2016-07-27T14:59:17.210304Z--2016-07-27T15:04:57.056134Z.mkv',
   'ts_end': '2016-07-27T15:04:57.056134Z',
   'ts_start': '2016-07-27T14:59:17.210304Z'},
  'stable': False,
  'ts_end': '2016-07-27T15:04:57.056134Z',
  'ts_start': '2016-07-27T14:59:16.877336Z'},
...
]
```

# Storing

In [15]:
def save_2_doc_n_locally(json_data, doc_path, filename):
    with open(filename, 'w') as fp:
        json.dump(json_data, fp, sort_keys=True, indent=2)
    
    with open(os.path.join(doc_path, filename), 'w') as fp:
        json.dump(json_data, fp, sort_keys=True, indent=2)

-------

-------

------

In [ ]:
def do_all(cam_id_left, cam_id_right, filename):
    sorted_data = df[((df.cam_id ==cam_id_left) | (df.cam_id == cam_id_right))]
    shared_intervals = get_start_of_shared_intervals(sorted_data, cam_id_left, cam_id_right, THOLD)
    extended_shared_intervals = extend_closest_videos(shared_intervals)
    shared_intervals_valid_json = extended_shared_intervals[::-1]
    extend_end_video_name(shared_intervals_valid_json, VIDEO_FILE_NAMES)
    converted = convert_2_valid_json(shared_intervals_valid_json)
    extend_overall_interval(converted)
    save_2_doc_n_locally(converted, DOC_PATH, filename)

In [ ]:
do_all(SIDE_A_LEFT_CAM,SIDE_A_RIGHT_CAM, SIDE_A_OUTPUT)
do_all(SIDE_B_LEFT_CAM,SIDE_B_RIGHT_CAM, SIDE_B_OUTPUT)